In [36]:
from fastbook import *
from IPython.display import display,HTML

In [ ]:
#pip install fastbook

In [35]:
import fastbook
fastbook.setup_book()

In [1]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [2]:
 files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

'What could have been an excellent hostage movie was totally ruined by what '

In [7]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#195) ['What','could','have','been','an','excellent','hostage','movie','was','totally','ruined','by','what','apparently','looks','like','a','bored','director','...','there','were','so','many','directions','that','the','movie','could','have'...]


In [24]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#207) ['xxbos','xxmaj','what','could','have','been','an','excellent','hostage','movie','was','totally','ruined','by','what','apparently','looks','like','a','bored','director','…','there','were','so','many','directions','that','the','movie','could'...]


In [10]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [12]:
#??replace_rep

## Subword Tokenization

In [15]:
txts = L(o.open().read() for o in files[:2000])

In [16]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [20]:
subword(1000)

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=tmp/texts.out --vocab_size=1000 --model_prefix=tmp/spm --character_coverage=0.99999 --model_type=unigram --unk_id=9 --pad_id=-1 --bos_id=-1 --eos_id=-1 --minloglevel=2 --user_defined_symbols=▁xxunk,▁xxpad,▁xxbos,▁xxeos,▁xxfld,▁xxrep,▁xxwrep,▁xxup,▁xxmaj --hard_vocab_limit=false


'▁What ▁could ▁have ▁been ▁an ▁ex ce ll ent ▁ho st age ▁movie ▁was ▁totally ▁ ru in ed ▁by ▁what ▁apparent ly ▁look s ▁like ▁a ▁bo red ▁director ▁ ... ▁there ▁were ▁so ▁many ▁direction s ▁that ▁the'

In [21]:
subword(200)

'▁ W h a t ▁c o u l d ▁have ▁be en ▁ an ▁ e x ce ll en t ▁h o s t a g e ▁movie ▁was ▁to t al ly ▁ r u in ed'

## Numericalization with fastai

from tokens to integers

In [25]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#207) ['xxbos','xxmaj','what','could','have','been','an','excellent','hostage','movie'...]

In [26]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

'(#1952) [\'xxunk\',\'xxpad\',\'xxbos\',\'xxeos\',\'xxfld\',\'xxrep\',\'xxwrep\',\'xxup\',\'xxmaj\',\'the\',\'.\',\',\',\'a\',\'and\',\'of\',\'to\',\'it\',\'is\',\'i\',\'"\'...]'

In [27]:
nums = num(toks)[:20]; nums

TensorText([   0,   84,   38,   92,   61,  496,  935,   26,   23,  458,    0,
              46,   60,  789,  382,   48,   12, 1441,  166,    0])

In [29]:
' '.join(num.vocab[o] for o in nums)

'xxunk could have been an excellent hostage movie was totally xxunk by what apparently looks like a bored director xxunk'

In [37]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6, 15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [39]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [40]:
nums200 = toks200.map(num)

In [41]:
dl = LMDataLoader(nums200)

In [43]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [44]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj what could have been an excellent hostage movie was totally xxunk by what apparently looks like a bored'

In [45]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj what could have been an excellent hostage movie was totally xxunk by what apparently looks like a bored director'

# Train

In [46]:
get_imdb = partial(get_text_files, folders=['train','test','unsup'])

In [47]:
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path,is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path,path=path,bs=128,seq_len=80)

In [48]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj the life and career ( the high - highs and low downs ) of versatile , charismatic actress - singer - dancer xxmaj judy xxmaj garland is profiled in this informative , well - researched and produced segment for "" american xxmaj masters "" . xxmaj while eventually not quite the thorough examination of the iconic performer one might have hoped for , this comes as close to revealing the real xxmaj judy as any other film biography","xxmaj the life and career ( the high - highs and low downs ) of versatile , charismatic actress - singer - dancer xxmaj judy xxmaj garland is profiled in this informative , well - researched and produced segment for "" american xxmaj masters "" . xxmaj while eventually not quite the thorough examination of the iconic performer one might have hoped for , this comes as close to revealing the real xxmaj judy as any other film biography to"
1,"for years before . xxmaj xxunk xxmaj xxunk ( bayside xxmaj shakedown ) is but the latest attempt to bring a popular xxmaj japanese xxup tv show to the big screen . xxmaj based on xxmaj fuji tv 's popular and hip cop show starring pretty boy xxmaj yuji xxmaj oda , xxmaj bayside xxmaj shakedown basically delivers what fans of the xxup tv show expect and want . xxmaj all the formulas that made the xxup tv show such","years before . xxmaj xxunk xxmaj xxunk ( bayside xxmaj shakedown ) is but the latest attempt to bring a popular xxmaj japanese xxup tv show to the big screen . xxmaj based on xxmaj fuji tv 's popular and hip cop show starring pretty boy xxmaj yuji xxmaj oda , xxmaj bayside xxmaj shakedown basically delivers what fans of the xxup tv show expect and want . xxmaj all the formulas that made the xxup tv show such a"


In [51]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [52]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.011861,3.902602,0.300550,49.531181,16:55


In [53]:
learn.save('1epoch')

Path('/home/mk8s/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
#The model not including the final layer is called the encoder. 
# We can save it with save_encoder:
learn.save_encoder('finetuned')

# Text Generation

In [55]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]

In [56]:
print("\n".join(preds))

i liked this movie because i was a very high hopes for the young top 20 show and at the age of 12 . i was so disappointed in the first Mystery Science Theatre series because i found the series to be
i liked this movie because of the low rating of the movie , the rating of one movie , and the mood that it showed . i was impressed by the movie , and i like this movie with a consistent plot . i love


# Classifier

In [58]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path,vocab=dls_lm.vocab),CategoryBlock),
    get_y=parent_label,
    get_items=partial(get_text_files, folders=['train','test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [59]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj warning : xxmaj does contain spoilers . \n\n xxmaj open xxmaj your xxmaj eyes \n\n xxmaj if you have not seen this film and plan on doing so , just stop reading here and take my word for it . xxmaj you have to see this film . i have seen it four times so far and i still have n't made up my mind as to what exactly happened in the film . xxmaj that is all i am going to say because if you have not seen this film , then stop reading right now . \n\n xxmaj if you are still reading then i am going to pose some questions to you and maybe if anyone has any answers you can email me and let me know what you think . \n\n i remember my xxmaj grade 11 xxmaj english teacher quite well . xxmaj",pos
2,"xxbos i thought that xxup rotj was clearly the best out of the three xxmaj star xxmaj wars movies . i find it surprising that xxup rotj is considered the weakest installment in the xxmaj trilogy by many who have voted . xxmaj to me it seemed like xxup rotj was the best because it had the most profound plot , the most suspense , surprises , most xxunk the ending ) and definitely the most episodic movie . i personally like the xxmaj empire xxmaj strikes xxmaj back a lot also but i think it is slightly less good than than xxup rotj since it was slower - moving , was not as episodic , and i just did not feel as much suspense or emotion as i did with the third movie . \n\n xxmaj it also seems like to me that after reading these surprising reviews that",pos


In [60]:
nums_samp = toks200[:10].map(num)

In [61]:
nums_samp.map(len)

(#10) [207,528,150,195,170,165,294,209,165,182]

In [62]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [63]:
learn = learn.load_encoder('finetuned')

FileNotFoundError: [Errno 2] No such file or directory: '/home/mk8s/.fastai/data/imdb/models/finetuned.pth'